In [ ]:
! pip install scikit-plot mlxtend
! pip install -U scikit-learn
#!python -c "import sklearn; sklearn.show_versions()"

# Imports

In [ ]:
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import matplotlib.pyplot as plt

from keras import Model
from keras.layers import Lambda, Input, Dropout, Flatten, LSTM, Concatenate, Bidirectional, Conv1D, MaxPooling1D
from keras import backend as K
from keras.callbacks import TensorBoard
from time import time
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from time import time
import csv
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import matplotlib.pyplot as plot
import seaborn
from pandas import datetime
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from math import sqrt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import sys
from sklearn import linear_model
import matplotlib.pyplot as plt
from matplotlib import style
# import pandas as pd
# import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support

# Load data

In [ ]:
def return_shape(X_trainz,window):
  size=X_trainz.shape[0]#* X_trainz.shape[1]
  wind=window
  shape0=0
  for i in range(2,window):
    if size%i==0:
      #print(i,278343/i)
      x=size/i
      for f in range(2,window):
        if x%f==0 and x*f==size:
          shape0=x
          wind=f
          #print(x,f, x/f,'\n-------- \n')
  return int(shape0),wind

def return_shape2(X_trainz,window):
  size=X_trainz.shape[0]#* X_trainz.shape[1]
  wind=window
  shape0=0
  for i in range(2,window):
    if size%i==0:
      #print(i,278343/i)
      x=size/i
      for f in range(2,window):
        if x%f==0:
          shape0=size/f
          wind=f
          #print(x,f, x/f,'\n-------- \n')
  return int(shape0),wind

In [ ]:
#from sklearn.cross_validation import train_test_split
#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=4)
import pandas as pd
from sklearn import preprocessing

df  = pd.read_csv('/content/marwa.csv')
# Training2 = pd.read_csv('SmartFall Training.csv')
# df = pd.concat([Training2, Testing2], ignore_index=True)
#df.describe()

shape0,w1=return_shape2(df,25)
#hape0,w1=return_shape(Training,25)
print(df.shape,shape0,w1,shape0*w1,shape0*w1*3)
#Training[['ms_accelerometer_x']]

In [ ]:
df.describe()

In [ ]:
col=df.columns

df=df[col[1:]]
df[:2]

In [ ]:
df = df.replace(',', '', regex=True)
df = df[1:].astype(float)
print (df.dtypes)

# Old idea
Train and test split from files

In [ ]:
# col=Training2.columns
# total=int(Training2.shape[0]/w1)*w1
# total1=int(Testing2.shape[0]/w1)*w1
# Training=Training2.iloc[:total,:]
# Testing=Testing2.iloc[:total1,:]

# # split into train and test
# lab_enc = preprocessing.LabelEncoder()
# trainy = lab_enc.fit_transform(Training[col[-1]])
# testy  = lab_enc.fit_transform(Testing[col[-1]])
# #--------------------------------------------------------------------
# n_train = 100
# #y = to_categorical(y)
# trainX, testX = Training[col[:-1]], Testing[col[:-1]]
# #trainy, testy = to_categorical(Training[col[-1]]), to_categorical(Testing[col[-1]])

# sc = StandardScaler()
# trainX = sc.fit_transform(trainX)
# testX  = sc.fit_transform(testX)

# print(trainX.shape, testX.shape, trainy.shape ,testy.shape)

# New idea 
- normal split

In [ ]:
col=df.columns
X, y=df[col[1: ]], df[col[0]]
X = preprocessing.minmax_scale(X)
# split into train and test
lab_enc = preprocessing.LabelEncoder()
y = lab_enc.fit_transform(y)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.2, random_state=2)
print(trainX.shape, testX.shape, trainy.shape ,testy.shape)

# Models

In [ ]:
def build_model2(layers):
        d = 0.2
        #input_shape = X_train.shape
        model = Sequential()
        #Input_layer
        model.add(LSTM(3, input_shape=(layers[0], layers[1]), return_sequences=True))
        #model.add(Dropout(d))
        #LSTM_layer
        model.add(LSTM(30, input_shape=(layers[0], layers[1]), return_sequences=False))
        #model.add(Dropout(d))
        #model.add(Flatten())
        model.add(Dense(30, activation='relu'))        
        #model.add(Flatten())
        #Output_layer
        model.add(Dense(2, activation='softmax' ))
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model

def build_model3(layers,Nueron,N_layer):
        d = 0.2
        #input_shape = X_train.shape
        model = Sequential()

        # The Input Layer :
        model.add(LSTM(3, input_shape=(layers[0], layers[1]), return_sequences=True))
        #model.add(Dropout(d))
        # The Hidden Layers :
        for i in range(N_layer-1):
          model.add(LSTM(Nueron, input_shape=(layers[0], layers[1]), return_sequences=True))
          #model.add(Dropout(d))
        #model.add(Flatten())
        model.add(LSTM(Nueron, input_shape=(layers[0], layers[1]), return_sequences=False))
        model.add(Dense(Nueron, activation='relu'))        
        #model.add(Flatten())

        # The Output Layer :
        model.add(Dense(2, activation='softmax'))
        # Compile the network :
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model

#Setup the plotting

In [ ]:
get_ipython().magic('matplotlib inline')
style = seaborn.axes_style("whitegrid")
style["axes.grid"] = False
seaborn.set_style("whitegrid", style)

# Evaluate Metrics

In [ ]:
from sklearn.metrics import auc,confusion_matrix,precision_recall_curve,precision_score,recall_score,roc_curve 
from sklearn.metrics import accuracy_score,f1_score,fbeta_score
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
#from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
import scikitplot as skplt

def F_b(beta,recal,precision):
  return (1+beta**2)*((recal*precision)/(recal+(beta**2)*precision))

def plot_precision_recall_curve(model,testy2,yhat):
  # calculate precision-recall curve
  precision, recall, thresholds = precision_recall_curve(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1))
  # calculate F1 score
  f1 = f1_score(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1))
  probas = model.predict_proba(X_test2, batch_size=64)
  skplt.metrics.plot_precision_recall_curve(y_test, probas)
  plt.show()
  #disp = plot_precision_recall_curve(classifier, X_test, y_test)
  #disp.ax_.set_title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))

def plot_history(history,name_f):
  plt.plot(history.history['accuracy'], label='training accuracy')
  plt.plot(history.history['val_accuracy'], label='testing accuracy')
  plt.title('Accuracy')
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.legend()
  
  file_name=name_f+"_Accuracy.pdf"
  #dest1 = os.path.join(path, file_name)
  plt.savefig(file_name, bbox_inches="tight")
  plt.show()

def plot_history2(history,name_f):
  plt.plot(history.history['loss'], label='training loss')
  plt.plot(history.history['val_loss'], label='testing loss')
  plt.title('Loss')
  plt.xlabel('epochs')
  plt.ylabel('loss')
  plt.legend()
  
  file_name=name_f+"_Loss.pdf"
  #dest1 = os.path.join(path, file_name)
  plt.savefig(file_name, bbox_inches="tight")
  plt.show()

def plot_auc(model, testy2, yhat, name_f):
  # Binarize the output
  y = label_binarize(testy2, classes=[0, 1])
  n_classes = y.shape[1]
  y_test=testy2
  y_score=yhat
  # Compute ROC curve and ROC area for each class
  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
      roc_auc[i] = auc(fpr[i], tpr[i])

  # Compute micro-average ROC curve and ROC area
  fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
  roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
  plt.figure()
  lw = 2
  plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
  plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver operating characteristic example')
  plt.legend(loc="lower right")
  file_name=name_f+"_AUC_ROC.pdf"
  #dest1 = os.path.join(path, file_name)
  plt.savefig(file_name, bbox_inches="tight")
  plt.show()

def Evaluate_model(model,history,name_file,algo,time,score_table,X_test,y_test):
  #X_train, y_train, X_test, y_test
  testy2 = to_categorical(y_test)
  loss, acc = model.evaluate(X_test, testy2, verbose=0)
  print('Model Accuracy: %.3f' % acc)

  # evaluate model on test set np.argmax(y_pred, axis=1)
  yhat = model.predict(X_test)
  acc = accuracy_score(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1))
   
  precision, recall, f1, support = precision_recall_fscore_support(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1))
  precision=np.average(precision, weights=support)
  recall=np.average(recall, weights=support)
  f1=np.average(f1, weights=support)
  fb = fbeta_score(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1), average='micro', beta=3)

  score_table.loc[algo,:] = time, acc, precision,recall,f1, F_b(3,recall,precision),fb
  CM = confusion_matrix(np.argmax(testy2, axis=1), np.argmax(yhat, axis=1))
  fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
  plt.show()

  plot_precision_recall_curve(model,testy2,yhat)
  plot_auc(model,testy2,yhat,name_file)
  # print(history.history.keys())
  plot_history(history,name_file)
  plot_history2(history,name_file)

# Data split

In [ ]:
from numpy import array
# (NumberOfExamples, TimeSteps, FeaturesPerStep)
# X_train = train_inputs.reshape((split,3,2))
# X_test = X_test.reshape((test_inputs.shape[0], 3, 2))
# you have 3 steps and 2 features
#TimeSteps
window = 1
X_train, y_train, X_test, y_test  = trainX, trainy, testX, testy

X_train2 = np.reshape(array(X_train), (int(X_train.shape[0]/window), window, X_train.shape[1]))
X_test2  = np.reshape(array(X_test), (int(X_test.shape[0]/window), window,  X_test.shape[1] ))

y_train=y_train[:int(X_train.shape[0]/window)]
y_test=y_test[:int(X_test.shape[0]/window)]
#X_train2 = array(X_train).reshape( X_train.shape[0],1, X_train.shape[1])
#X_test2 = array(X_test).reshape(X_test.shape[0],1, X_test.shape[1])
X_train2.shape,y_train.shape, X_test2.shape,y_test.shape

# Table to save final results

In [ ]:
# initialize a score table to log the performance of various algorithms
index = ['LSTM_'+str(n) for n in [20,30,50,80]]
score_table_2 = pd.DataFrame(index = index, 
                             columns= ['Time','Accuracy','precision','recall','F1','F_b','Fb_metric'])

# Models fitting

In [ ]:
from keras.utils.vis_utils import plot_model
import time

  
def fit_model(window1,Nueron1,N_layer1,X_train_f,y_train_f,X_test_f,y_test_f,Name1,path_model,score_table):
  layers=[window1,X_train_f.shape[2]]
  Nueron=Nueron1
  N_layer=N_layer1
  start_time = time.clock()

  model = build_model3(layers,Nueron,N_layer)
  y_train2 = to_categorical(y_train_f)
  print("X_train_f: ",X_train_f.shape)
  history = model.fit(X_train_f,y_train2,batch_size=20, epochs=500,validation_split=0.1,verbose=0)
  
  # save the model
  path=path_model
  name_file=Name1+'_N_'+str(N_layer1)+'_L'
  name_file1=os.path.join(path, name_file)
  plot_model(model, show_shapes=True, to_file=name_file1+'_model.png')
  model.save(name_file1+'_model.h5')
  # summarize
  print(model.summary())
  time1 = time.clock() - start_time
  Evaluate_model(model,history,name_file1,Name1,time1,score_table,X_test_f,y_test_f)


# Results Summary- Single DL model

In [ ]:
import os
import shutil

try:
  os.mkdir('Single DL model')
  #os.mkdir('classification2')
except:
  shutil.rmtree('Single DL model')
  #shutil.rmtree('classification2')
  os.mkdir('Single DL model')
  #os.mkdir('classification2')

In [ ]:
window1=1
N_layer1=1
path_model='./Single DL model/'
for n in [90,80]:
  Name1="LSTM_"+str(n)
  fit_model(window1,n,N_layer1,X_train2,y_train,X_test2,y_test,Name1,path_model,score_table_2)

In [ ]:
score_table_2

# Results Summary- Single DL + additional layer model

In [ ]:
# initialize a score table to log the performance of various algorithms
index = ['LSTM_'+str(n)+'| Added Layer' for n in [20,30,50,80]]
score_table_3 = pd.DataFrame(index = index, 
                             columns= ['Time','Accuracy','precision','recall','F1','F_b','Fb_metric'])

try:
  os.mkdir('Single additional layer model')
except:
  shutil.rmtree('Single additional layer model')
  os.mkdir('Single additional layer model')

In [ ]:
window1=1
N_layer1=2
path_model='./Single additional layer model/'
for n in [20,30,50,80]:
  Name1="LSTM_"+str(n)+'| Added Layer'
  fit_model(window1,n,N_layer1,X_train2,y_train,X_test2,y_test,Name1,path_model,score_table_3)

In [ ]:
score_table_3

# Bagging

In [ ]:
# load all models
#X_train2,y_train,X_test2,y_test
n_members = 4
path='/content/Single additional layer model/'
members = load_all_models(n_members,path)
print('Loaded %d models' % len(members))
# evaluate standalone models on test dataset
for model in members:
	testy_enc = to_categorical(y_test)
	_, acc = model.evaluate(X_test2, testy_enc, verbose=0)
	print('Model Accuracy: %.3f' % acc)
# fit stacked model using the ensemble
model,history = fit_stacked_model(members, X_test2, testy)
# evaluate model on test set
yhat = stacked_prediction(members, model, X_test2)
acc = accuracy_score(testy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

In [ ]:
model

In [ ]:
from sklearn.ensemble import BaggingClassifier
bp = BaggingClassifier(model,n_estimators=10)
bp=bp.fit(X_train,y_train)
acc = bp.score(X_test, y_test)
print('Model Accuracy: %.3f' % acc)

# Results Summary- Ensemble DL with Boosting

As we discussed, bagging is particularly effective for individual high-variance classifiers because the final majority-vote tends to smooth out the individual classifiers and produce a more stable collaborative solution. On the other hand, boosting is particularly effective for high-bias classifiers that are slow to adjust to new data. On the one hand, boosting is similiar to bagging in that it uses a majority-voting (or averaging for numeric prediction) process at the end; and it also combines individual classifiers of the same type. On the other hand, boosting is serially iterative, whereas the individual classifiers in bagging can be trained in parallel. Boosting uses the misclassifications of prior iterations to influence the training of the next iterative classifier by weighting those misclassifications more heavily in subsequent steps. This means that, at every step, boosting focuses more and more on specific misclassifications up to that point, letting the prior classifications be carried by earlier iterations.

The primary implementation for boosting in Scikit-learn is the Adaptive Boosting (AdaBoost) algorithm, which does classification (AdaBoostClassifier) and regression (AdaBoostRegressor). The first step in the basic AdaBoost algorithm is to initialize the weights over each of the training set indicies, $D_0(i)=1/n$ where there are $n$ elements in the training set. Note that this creates a discrete uniform distribution over the indicies, not over the training data $\lbrace (x_i,y_i) \rbrace$ itself. In other words, if there are repeated elements in the training data, then each gets its own weight. The next step is to train the base classifer $h_k$ and record the classification error at the $k^{th}$ iteration, $\epsilon_k$. Two factors can next be calculated using $\epsilon_k$,
$$ \alpha_k = \frac{1}{2}\log \frac{1-\epsilon_k}{\epsilon_k} $$

and the normalization factor,
$$ Z_k = 2 \sqrt{ \epsilon_k (1- \epsilon_k) } $$

For the next step, the weights over the training data are updated as in the following,
$$ D_{k+1}(i) = \frac{1}{Z_k} D_k(i)\exp{(-\alpha_k y_i h_k(x_i))} $$

The final classification result is assembled using the $\alpha_k$ factors, $g = \sgn(\sum_{k} \alpha_k h_k)$.

To re-do the problem above using boosting with perceptrons, we set up the AdaBoost classifier in the following,

- create models for stacking

In [ ]:
# initialize a score table to log the performance of various algorithms
index = ['LSTM_'+str(n)+'| Added Layer' for n in [20,30,50,80]]
score_table_4 = pd.DataFrame(index = index, 
                             columns= ['Time','Accuracy','precision','recall','F1','F_b','Fb_metric'])

try:
  os.mkdir('Ensemble DL with Boosting')
except:
  shutil.rmtree('Ensemble DL with Boosting')
  os.mkdir('Ensemble DL with Boosting')

In [ ]:
window1=1
N_layer1=2
path_model='./Ensemble DL with Boosting/'
no_models=[10,15,50]
for n in [5,10,15,30]:
  Name1=str()+"LSTM |"+str(n)+'Neuron'
  fit_model(window1,n,N_layer1,X_train2,y_train,X_test2,y_test,Name1,path_model,score_table_4)

In [ ]:
score_table_4

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf=AdaBoostClassifier(model,n_estimators=3,
                       algorithm='SAMME',
                       learning_rate=0.5)
clf=clf.fit(X_train,y_train)
acc = clf.score(X_test, y_test)
print('Model Accuracy: %.3f' % acc)

# Results Summary- Ensemble DL with Stacking 

# Stacking function
- Load Model
- Save Model
- Stacked data

In [ ]:
from keras.models import load_model
from sklearn.linear_model import LogisticRegression
# load models from file
def load_all_models(n_models,path):
  all_models = list()
  names=['LSTM_'+str(n)+'| Added Layer_N_2_L' for n in [20,30,50,80]]
  for i in range(n_models):
    # define filename for this ensemble
    filename = path+str(names[i])+'_model.h5'
    # load model from file
    model = load_model(filename)
    # add to list of members
    all_models.append(model)
    print('>loaded %s' % filename)
  return all_models

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

# fit a model based on the outputs from the ensemble members
# create new training data set from previous 
# fit standalone model # final model
# eroor testy dim 2 --> dim 1 using inverse to_categorical
def fit_stacked_model(members, inputX, inputy):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	model   = LogisticRegression()
	inputy2 = lab_enc.inverse_transform(inputy)#[np.argmax(inputy, axis=None, out=None) for y in inputy]
	print('fit_stacked_model: ',stackedX.shape,inputy2.shape)
	history = model.fit(stackedX, inputy2.reshape(-1,1))
	return model,history

# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

def Evaluate_model2(model,algo,time,score_table,X_test,testy2):
   # evaluate model on test set np.argmax(y_pred, axis=1)
  yhat = model.predict(X_test)
  acc = accuracy_score(testy2, yhat)
  recall1 = recall_score(testy2, yhat)
  print('Model Test Accuracy: %.3f' % acc)
  precision, recall, f1, support = precision_recall_fscore_support(testy2, yhat)
  #precision, recall, thresholds = precision_recall_curve(testy2, yhat)
  # calculate F1 score 
  #f1 = f1_score(testy2, yhat)
  precision=np.average(precision, weights=support)
  recall=np.average(recall, weights=support)
  f1=np.average(f1, weights=support)
  fb = fbeta_score(testy2, yhat, average='micro', beta=3)

  score_table.loc[algo,:] = time, acc, precision,recall,f1, F_b(3,recall,precision),fb
  CM = confusion_matrix(testy2, yhat)
  fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
  plt.show()

- overlab data 

In [ ]:
m='5'+'2'
k=16
z=2.5
print(m)

In [ ]:
import math
def assign_set2(no_models,m_overlap,list_models,start,end,data_size):
  rows_per_model=round(data_size/((m_overlap-1)*(no_models-1)+1))#how many rows each model will take
  total1 = data_size/no_models
  total  = math.floor(total1)
  #first model
  list_models[start][0] = 0
  list_models[end][0]   = total+m_overlap

   #rest model
  for i in range(1,no_models):
     list_models[start][i] = total*i-(m_overlap)
     list_models[end][i]   = list_models[start][i]+total+2*m_overlap
   #last model
  list_models[start][no_models-1] = total*(no_models-1)-m_overlap
  list_models[end][no_models-1]   = data_size

no_models=80
m_overlap=5
data_size=183806
data_models=pd.DataFrame(index=range(no_models),
                         columns=['start_row','end_row','start_overlap','end_overlap'])
assign_set2(no_models,0,data_models,'start_row','end_row',data_size)
assign_set2(no_models,m_overlap,data_models,'start_overlap','end_overlap',data_size)
data_models[['start_overlap','end_overlap']].loc[0]

In [ ]:
import os
import shutil
# initialize a score table to log the performance of various algorithms
index = ['LSTM_'+str(m)+'M'+str(n)+'N' for m,n in zip([4,10,80],[30,15,6]) ]
score_table_5 = pd.DataFrame(index = index, 
                             columns= ['Time','Accuracy','precision','recall','F1','F_b','Fb_metric'])
try:
  os.mkdir('Ensemble Stacking')
except:
  shutil.rmtree('Ensemble Stacking')
  os.mkdir('Ensemble Stacking')

score_table_5

- Create models for stacking

In [ ]:
window1=1
N_layer1=2
no_models=[4,10,80]
for n,j in zip([30,15,6],no_models):
  Name1='LSTM_'+str(j)+'M'+str(n)+'N'
  path_model='./Ensemble Stacking/'+Name1
  os.mkdir(path_model)
  for k in range(j):
    Name1='LSTM_'+str(j)+'M'+str(n)+'N'+str(k)
    fit_model(window1,n,N_layer1,X_train2,y_train,X_test2,y_test,Name1,path_model,score_table_5)

In [ ]:
score_table_6=score_table_5
# load all models
no_models=[4,10,80]
for n,j in zip([30,15,6],no_models):
  Name1='LSTM_'+str(j)+'M'+str(n)+'N'
  path_model='./Ensemble Stacking/'+Name1
  #X_train2,y_train,X_test2,y_test
  n_members = j
  #path='/content/Single additional layer model/'
  members = load_all_models(n_members,path_model)
  print('Loaded %d models' % len(members))
  # evaluate standalone models on test dataset
  for model in members:
    testy_enc = to_categorical(y_test)
    _, acc = model.evaluate(X_test2, testy_enc, verbose=0)
    print('Model Accuracy: %.3f' % acc)
  # fit stacked model using the ensemble
  model,history = fit_stacked_model(members, X_test2, testy)
  # evaluate model on test set
  yhat = stacked_prediction(members, model, X_test2)
  acc = accuracy_score(testy, yhat)
  print('Stacked Test Accuracy: %.3f' % acc)
  Evaluate_model2(model,Name1,time,score_table_6,X_test,testy2)

In [ ]:
score_table_6

- download files from colab to local drive

In [ ]:
!zip -r /content/Ensemble Stacking.zip /content/Ensemble Stacking
#!zip -r /content/classification2.zip /content/classification2

from google.colab import files
files.download("/content/Ensemble Stacking.zip")
#files.download("/content/classification2.zip")

 - divide data based on Overlap

In [ ]:
def overlab_data(no_models,m_overlap,data_size):
  data_models=pd.DataFrame(index=range(no_models),
                         columns=['start_row','end_row','start_overlap','end_overlap'])
  assign_set2(no_models,0,data_models,'start_row','end_row',data_size)
  assign_set2(no_models,m_overlap,data_models,'start_overlap','end_overlap',data_size)
  return data_models#[['start_overlap','end_overlap']].loc[0]

In [ ]:
window1=1
N_layer1=2
no_models=[4,10,80]
for n,j in zip([30,15,6],no_models):
  Name1='LSTM_'+str(j)+'M'+str(n)+'N'
  path_model='./Ensemble Stacking Overlab/'+Name1
  no_models=j
  m_overlap=5
  data_size=X_train2.shape[0]
  od=overlab_data(no_models,m_overlap,data_size)
  for k in range(j):
    start,end=od[['start_overlap','end_overlap']].loc[k]
    Name1='LSTM_'+str(j)+'M'+str(n)+'N'+str(k)
    fit_model(window1,n,N_layer1,X_train2[start:end,:],y_train[start:end,:],X_test2,y_test,Name1,path_model,score_table_5)

In [ ]:
score_table_7=score_table_5
# load all models
no_models=[4,10,80]
for n,j in zip([30,15,6],no_models):
  Name1='LSTM_'+str(j)+'M'+str(n)+'N'
  path_model='./Ensemble Stacking/'+Name1
  #X_train2,y_train,X_test2,y_test
  n_members = j
  #path='/content/Single additional layer model/'
  members = load_all_models(n_members,path_model)
  print('Loaded %d models' % len(members))
  # evaluate standalone models on test dataset
  for model in members:
    testy_enc = to_categorical(y_test)
    _, acc = model.evaluate(X_test2, testy_enc, verbose=0)
    print('Model Accuracy: %.3f' % acc)
  # fit stacked model using the ensemble
  model,history = fit_stacked_model(members, X_test2, testy)
  # evaluate model on test set
  yhat = stacked_prediction(members, model, X_test2)
  acc = accuracy_score(testy, yhat)
  print('Stacked Test Accuracy: %.3f' % acc)
  Evaluate_model2(model,Name1,time,score_table_6,X_test,testy2)

#LSTM ATTENTION

In [ ]:
import keras
from keras.models import Model,Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Activation,Dropout,Conv1D,GlobalMaxPooling1D,GRU,Bidirectional,LSTM,SimpleRNN
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import backend as K 
from keras.utils import to_categorical

from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.plotting import plot_decision_regions
from mlxtend.plotting import plot_confusion_matrix

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn import model_selection


In [ ]:
import os
import shutil

try:
  os.mkdir('LSTM ATTENTION')
  #os.mkdir('classification2')
except:
  shutil.rmtree('LSTM ATTENTION')
  #shutil.rmtree('classification2')
  os.mkdir('LSTM ATTENTION')
  #os.mkdir('classification2')

In [ ]:
def fit_model_L_A(layers,optimizer='adam'):
  #input_shape = X_train.shape
  model = Sequential()

  # The Input Layer :
  activations=LSTM(3, input_shape=(layers[0], layers[1]), return_sequences=True)
  #model.add(Dropout(d))
  # The Hidden Layers :
  # compute importance for each step
  attention = Dense(1, activation='tanh')(activations)
  attention = keras.layers.Flatten()(attention)
  attention = keras.layers.Activation('softmax')(attention)
  attention = keras.layers.RepeatVector(units)(attention)
  attention = keras.layers.Permute([2, 1])(attention)

  multiply_layer = keras.layers.Multiply()
  sent_representation = multiply_layer([activations, attention])
  # sent_representation = keras.layers.Multiply([activations, attention])
  sent_representation = keras.layers.Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(units,))(sent_representation)

  probabilities = Dense(1, activation='sigmoid')(sent_representation)
  model_L_A = Model(input=input_, output=probabilities)
  model_L_A.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

  return model_L_A


model_L_A = KerasClassifier(build_fn=fit_model_L_A,epochs=3,batch_size=64, verbose=2)
  
model_L_A_param_grid = {'optimizer':['adam']}

model_L_A_grid = GridSearchCV(model_L_A,cv=KFold(n_splits=2),param_grid=model_L_A_param_grid,
                    return_train_score=True, scoring=['accuracy'],refit='accuracy')

model_L_A_summary = fit_model_L_A([3,1])
print(model_L_A_summary.summary())
grid_results = model_L_A_grid.fit(X_train,Y_train)


- StackingClassifier
- Running GridSearch Models
- Overlap Technique

In [ ]:
model_BG_grid_results = model_BG_grid.fit(X_train1, y_train1)
model_BL_grid_results = model_BL_grid.fit(X_train2, y_train2)
model_BR_grid_results = model_BR_grid.fit(X_train3, y_train3)
model_L_grid_results = model_L_grid.fit(X_train, y_train)
model_SR_grid_results = model_SR_grid.fit(X_train, y_train)
model_GRU_grid_results = model_GRU_grid.fit(X_train, y_train)
model_L_A_grid_results = model_L_A_grid.fit(X_train, y_train)
model_text_CNN_grid_results = model_text_CNN_grid.fit(X_train, y_train)

In [ ]:
clf1 = grid_results
clf2 = RandomForestClassifier(random_state=1, n_estimators=300)
clf3 = GaussianNB()
# Logit will be used for stacking
lr = LogisticRegression(solver='lbfgs')
sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3], meta_classifier=lr, use_probas=True, cv=2)

# Do CV
for clf, label in zip([clf1, clf2, clf3, sclf], 
                      ['grid_results', 
                       'Random Forest', 
                       'Naive Bayes',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X_train, y_train, cv=2, scoring='roc_auc')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

# Fit on train data / predict on test data
sclf_fit = sclf.fit(X_train, y_train)
mypreds = sclf_fit.predict_proba(X_test)

# Machine Learning Models

In [ ]:
from datetime import datetime
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        tmin, tsec = divmod((datetime.now() - start_time).total_seconds(), 60)
        print(' Time taken: %i minutes and %s seconds.' %(tmin, round(tsec, 2)))

In [ ]:
from sklearn.metrics.classification import precision_recall_fscore_support
def Evaluate_model2(model,algo,time,score_table,X_test,testy2):
   # evaluate model on test set np.argmax(y_pred, axis=1)
  yhat = model.predict(X_test)
  acc = accuracy_score(testy2, yhat)
  recall1 = recall_score(testy2, yhat)
  print('Model Test Accuracy: %.3f' % acc)
  precision, recall, f1, support = precision_recall_fscore_support(testy2, yhat)
  #precision, recall, thresholds = precision_recall_curve(testy2, yhat)
  # calculate F1 score 
  #f1 = f1_score(testy2, yhat)
  precision=np.average(precision, weights=support)
  recall=np.average(recall, weights=support)
  f1=np.average(f1, weights=support)
  fb = fbeta_score(testy2, yhat, average='micro', beta=3)

  score_table.loc[algo,:] = time, acc, precision,recall,f1, F_b(3,recall,precision),fb
  CM = confusion_matrix(testy2, yhat)
  fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
  plt.show()

# initialize a score table to log the performance of various algorithms
index = ['xgb','SVC','AdaBoostClassifier','BaggingClassifier',
         'RandomForestClassifier','StackingClassifier']
score_table_ml = pd.DataFrame(index = index, 
                             columns= ['Time','Accuracy','precision','recall','F1','F_b','Fb_metric'])

In [ ]:
# fit model on dataset
def fit_model(trainX, trainy):
	# define model
	model = Sequential()
	model.add(Dense(25, input_dim=3, activation='relu'))
	model.add(Dense(2, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit model
	model.fit(trainX, trainy, epochs=500, verbose=0)
	return model


In [ ]:
def ml(model,name_clf,X_test,y_test,score_table_ml):
  start_time = datetime.now()
  
  model.fit(X_test,y_test)
  time=datetime.now()-start_time
  Evaluate_model2(model,name_clf,time,score_table_ml,X_test,y_test)
  print(classification_report_imbalanced(y_test, model.predict(X_test)))

xgb=XGBClassifier()
#SVC1= SVC(kernel='rbf', C=1e3,random_state=42)
#SVC1.fit(X_test,y_test)
Baggingclf = BaggingClassifier(base_estimator=xgb,n_estimators=10, random_state=0)
Adaclf = AdaBoostClassifier(n_estimators=100, random_state=0)
rfc= RandomForestClassifier(n_estimators=100,random_state=42)
#rfc.fit(X_train, y_train)

estimators = [('rf',  RandomForestClassifier(n_estimators=10, random_state=42)),
              ('Ada', AdaBoostClassifier(n_estimators=100, random_state=0))]

stc = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())   

classifier = [xgb,  Adaclf, Baggingclf, rfc, stc]
index = ['xgb', 'AdaBoostClassifier','BaggingClassifier','RandomForestClassifier','StackingClassifier']

for m,name_clf in zip(classifier,index):
  print('\n ========'+name_clf+'================= \n')
  ml(m,name_clf,X_test,y_test,score_table_ml)
  print('\n ========================= \n')

In [ ]:
get_ipython().magic('matplotlib inline')
style = seaborn.axes_style("whitegrid")
style["axes.grid"] = False
seaborn.set_style("whitegrid", style)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve
#xgb,  Adaclf, Baggingclf, rfc, stc
xgb_disp  = plot_precision_recall_curve(xgb, X_test, y_test)
Adaclf_disp  = plot_precision_recall_curve(Adaclf, X_test, y_test, ax=xgb_disp.ax_)
Baggingclf_disp = plot_precision_recall_curve(Baggingclf, X_test, y_test, ax=Adaclf_disp.ax_)
rfc_disp = plot_precision_recall_curve(rfc, X_test, y_test, ax=Baggingclf_disp.ax_)
clf2_disp = plot_precision_recall_curve(stc, X_test, y_test, ax=rfc_disp.ax_)
clf2_disp.figure_.suptitle("Precision-Recall comparison")
plt.xlabel('Recall')
plt.ylabel('Precision')
#plt.title('Precision-Recall comparison')
plt.legend(prop=dict(size=14))
plt.show()
plt.show()

In [ ]:
score_table_ml

# Binary classification

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
#from sklearn.metrics import plot_roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt

start_time = timer(None)

#X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = trainX, testX, trainy, testy 

svc = SVC(random_state=42)
svc.fit(X_train, y_train)
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

clf1 = AdaBoostClassifier(n_estimators=100, random_state=0)
clf1.fit(X_train, y_train)

estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
              ('svr', make_pipeline(StandardScaler(),SVC(random_state=42)))]

clf2 = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())             

clf2.fit(X_train, y_train).score(X_test, y_test)

from sklearn.metrics import average_precision_score
y_score1 = svc.predict(X_test)
y_score2 = rfc.predict(X_test)
y_score3 = clf1.predict(X_test)
y_score4 = clf2.predict(X_test)

average_precision1 = average_precision_score(y_test, y_score1)
average_precision3 = average_precision_score(y_test, y_score3)
average_precision4 = average_precision_score(y_test, y_score4)
print('Average precision-recall score: {0:0.2f}'.format(average_precision1))
print('Average precision-recall score: {0:0.2f}'.format(average_precision3))
print('Average precision-recall score: {0:0.2f}'.format(average_precision4))

timer(start_time)

In [ ]:
def plot_p_recall2(names,precision,recall,average_precision):
  # setup plot details
  colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])
  plt.figure(figsize=(7, 8))
  
  lines = []
  labels = []
  for i, color in zip(range(len(names)), colors):
      l, = plt.plot(recall[i]['micro'], precision[i]['micro'],marker='o', linestyle='dashed', color=color ,lw=2 )
      lines.append(l)
      labels.append('{0} (area = {1:0.2f})'.format(names[i], average_precision[i]['micro']))

  fig = plt.gcf()
  fig.subplots_adjust(bottom=0.25)
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('Recall')
  plt.ylabel('Precision')
  plt.title('Precision-Recall comparison')
  plt.legend(lines, labels, prop=dict(size=14))
  plt.show()

precision=[]
recall=[]
average_precision=[]
for i in [y_score2,y_score3,y_score4]:
  precision1,recall1,average_precision1=Pre_Recall(i)
  precision.append(precision1)
  recall.append(recall1)
  average_precision.append(average_precision1)

plot_p_recall2(names,precision,recall,average_precision)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve

svc_disp  = plot_precision_recall_curve(svc, X_test, y_test)
rfc_disp  = plot_precision_recall_curve(rfc, X_test, y_test, ax=svc_disp.ax_)
clf1_disp = plot_precision_recall_curve(clf1, X_test, y_test, ax=rfc_disp.ax_)
clf2_disp = plot_precision_recall_curve(clf2, X_test, y_test, ax=clf1_disp.ax_)
clf2_disp.figure_.suptitle("Precision-Recall comparison")
plt.xlabel('Recall')
plt.ylabel('Precision')
#plt.title('Precision-Recall comparison')
plt.legend(prop=dict(size=14))
plt.show()
plt.show()

# Multi-label classification

In [ ]:
from sklearn.preprocessing import label_binarize
start_time = timer(None)

# Use label_binarize to be multi-label like settings
Y = label_binarize(y, classes=[0, 1, 2])
n_classes = Y.shape[1]

# Split into training and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.5,random_state=7)

# We use OneVsRestClassifier for multi-label prediction
from sklearn.multiclass import OneVsRestClassifier

# Run classifier
# SVC = SVC(random_state=7)
# SVC.fit(X_train, y_train)
# y_score1 = SVC.decision_function(X_test)

rfc = OneVsRestClassifier(RandomForestClassifier(random_state=42))
rfc.fit(X_train, y_train)
y_score2 = rfc.predict(X_test)

clf1 = OneVsRestClassifier(AdaBoostClassifier(n_estimators=100, random_state=0))
clf1.fit(X_train, y_train)
y_score3 = clf1.predict(X_test)

estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
              ('rf1', RandomForestClassifier(n_estimators=20, random_state=42))]

clf2 = OneVsRestClassifier(StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()))
clf2.fit(X_train, y_train).score(X_test, y_test)
y_score4 = clf2.predict(X_test)

names=['RandomForestClassifier','AdaBoostClassifier','StackingClassifier']

timer(start_time)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import numpy as np
from itertools import cycle

def Pre_Recall(y_score1):
  # For each class
  precision = dict()
  recall = dict()
  average_precision = dict()
  for i in range(n_classes):
      precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],y_score1[:, i])
      average_precision[i] = average_precision_score(y_test[:, i], y_score1[:, i])

  # A "micro-average": quantifying score on all classes jointly
  precision["micro"], recall["micro"], _ = precision_recall_curve(y_test.ravel(),y_score1.ravel())
  average_precision["micro"] = average_precision_score(y_test, y_score1, average="micro")
  print('Average precision score, micro-averaged over all classes: {0:0.2f}'.format(average_precision["micro"]))
  return precision,recall,average_precision

def plot_p_recall2(names,precision,recall,average_precision):
  # setup plot details
  colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])
  plt.figure(figsize=(7, 8))
  
  lines = []
  labels = []
  for i, color in zip(range(len(names)), colors):
      l, = plt.plot(recall[i]['micro'], precision[i]['micro'],marker='o', linestyle='dashed', color=color ,lw=2 )
      lines.append(l)
      labels.append('{0} (area = {1:0.2f})'.format(names[i], average_precision[i]['micro']))

  fig = plt.gcf()
  fig.subplots_adjust(bottom=0.25)
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('Recall')
  plt.ylabel('Precision')
  plt.title('Precision-Recall comparison')
  plt.legend(lines, labels, prop=dict(size=14))
  plt.show()


In [ ]:
precision=[]
recall=[]
average_precision=[]
for i in [y_score2,y_score3,y_score4]:
  precision1,recall1,average_precision1=Pre_Recall(i)
  precision.append(precision1)
  recall.append(recall1)
  average_precision.append(average_precision1)

plot_p_recall2(names,precision,recall,average_precision)

In [ ]:
average_precision#[0]['micro']

#Comparison of ensembling classifiers internally using sampling

In [ ]:
from collections import Counter
from imblearn.under_sampling import NearMiss
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced

print('Training target statistics: {}'.format(Counter(y_train)))
print('Testing target statistics: {}'.format(Counter(y_test)))

rf = RandomForestClassifier(n_estimators=50, random_state=0, n_jobs=-1)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Classify and report the results
print(classification_report_imbalanced(y_test, rf.predict(X_test)))

In [ ]:
import itertools

import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

#from imblearn.datasets import fetch_datasets
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import RUSBoostClassifier

from imblearn.metrics import geometric_mean_score


def plot_confusion_matrix(cm, classes, ax, normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    print(cm)
    print('')

    ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.set_title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.sca(ax)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        ax.text(j, i, format(cm[i, j], fmt),horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")

    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)
print('Decision tree classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_tree),geometric_mean_score(y_test, y_pred_tree)))
cm_tree = confusion_matrix(y_test, y_pred_tree)
fig, ax = plt.subplots()
plot_confusion_matrix(cm_tree, classes=np.unique([0,1]), ax=ax, title='Decision tree')

In [ ]:
bagging = BaggingClassifier(n_estimators=50, random_state=0, n_jobs=-1)
balanced_bagging = BalancedBaggingClassifier(n_estimators=50, random_state=0,n_jobs=-1)

bagging.fit(X_train, y_train)
alanced_bagging.fit(X_train, y_train)

y_pred_bc = bagging.predict(X_test)
y_pred_bbc = balanced_bagging.predict(X_test)

print('Bagging classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_bc),geometric_mean_score(y_test, y_pred_bc)))
cm_bagging = confusion_matrix(y_test, y_pred_bc)
fig, ax = plt.subplots(ncols=2)
plot_confusion_matrix(cm_bagging, classes=np.unique([0,1]), ax=ax[0], title='Bagging')

print('Balanced Bagging classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_bbc), geometric_mean_score(y_test, y_pred_bbc)))
cm_balanced_bagging = confusion_matrix(y_test, y_pred_bbc)
plot_confusion_matrix(cm_balanced_bagging, classes=np.unique([0,1]),ax=ax[1], title='Balanced bagging')

In [ ]:
rf = RandomForestClassifier(n_estimators=50, random_state=0, n_jobs=-1)
brf = BalancedRandomForestClassifier(n_estimators=50, random_state=0,
                                     n_jobs=-1)

rf.fit(X_train, y_train)
brf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_pred_brf = brf.predict(X_test)

# Similarly to the previous experiment, the balanced classifier outperform the
# classifier which learn from imbalanced bootstrap samples. In addition, random
# forest outsperforms the bagging classifier.

print('Random Forest classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_rf),
              geometric_mean_score(y_test, y_pred_rf)))
cm_rf = confusion_matrix(y_test, y_pred_rf)
fig, ax = plt.subplots(ncols=2)
plot_confusion_matrix(cm_rf, classes=np.unique([0,1]), ax=ax[0],
                      title='Random forest')

print('Balanced Random Forest classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_brf),
              geometric_mean_score(y_test, y_pred_brf)))
cm_brf = confusion_matrix(y_test, y_pred_brf)
plot_confusion_matrix(cm_brf, classes=np.unique([0,1]), ax=ax[1],
                      title='Balanced random forest')

In [ ]:
base_estimator = AdaBoostClassifier(n_estimators=10)
eec = EasyEnsembleClassifier(n_estimators=10,base_estimator=base_estimator)
eec.fit(X_train, y_train)
y_pred_eec = eec.predict(X_test)
print('Easy ensemble classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_eec), geometric_mean_score(y_test, y_pred_eec)))
cm_eec = confusion_matrix(y_test, y_pred_eec)
fig, ax = plt.subplots(ncols=2)
plot_confusion_matrix(cm_eec, classes=np.unique([0,1]), ax=ax[0],title='Easy ensemble classifier')

rusboost = RUSBoostClassifier(n_estimators=10, base_estimator=base_estimator)
rusboost.fit(X_train, y_train)
y_pred_rusboost = rusboost.predict(X_test)
print('RUSBoost classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_rusboost),geometric_mean_score(y_test, y_pred_rusboost)))
cm_rusboost = confusion_matrix(y_test, y_pred_rusboost)
plot_confusion_matrix(cm_rusboost, classes=np.unique([0,1]),ax=ax[1], title='RUSBoost classifier')

plt.show()